In [1]:
import pandas as pd
from sklearn.metrics import *
from tqdm import tqdm
from utils import Hetero_Data_Processor_Filter_on_Test_since_first_post
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn

import torch_geometric.transforms as T
from torch_geometric.datasets import DBLP
from torch_geometric.nn import HANConv, Linear


In [2]:

# Usage
file_path_replies = r"replies_charlie_hebdo.pkl"
file_path_posts = r"posts_charlie_hebdo.pkl"



processor = Hetero_Data_Processor_Filter_on_Test_since_first_post(file_path_replies, file_path_posts, time_cut=3*60*24)
data = processor.process()


/workspaces/rumour-detection-gnn/utils.py:1083: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_val_reply_features[['reply_followers','time_diff']] = scaler_replies.transform(test_val_reply_features[['reply_followers','time_diff']])


In [3]:
data

HeteroData(
  id={
    x=[2002, 106],
    y=[2002],
    train_mask=[2002],
    val_mask=[2002],
    test_mask=[2002],
  },
  reply_user_id={ x=[18966, 104] },
  (id, retweet, reply_user_id)={ edge_index=[2, 18966] },
  (reply_user_id, rev_retweet, id)={ edge_index=[2, 18966] }
)

In [4]:
class HAN(nn.Module):
    def __init__(self, dim_in, dim_out, dim_h=64, heads=4):
        super().__init__()
        self.han = HANConv(dim_in, dim_h, heads=heads,dropout=0.2, metadata=data.metadata())
        self.han2 = HANConv(dim_h, dim_h, heads=heads, dropout=0.2, metadata=data.metadata())
        self.linear = nn.Linear(dim_h, dim_out)

    def forward(self, x_dict, edge_index_dict):
        out = self.han(x_dict, edge_index_dict)
        out = self.han2(out, edge_index_dict)
        out = self.linear(out['id'])
        return out
    

In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
import torch
from torch.nn.functional import cross_entropy

def evaluate(model, data, mask_names):
    model.eval()
    out = model(data.x_dict, data.edge_index_dict)
    preds = out.argmax(dim=-1)
    labels = data['id'].y

    if isinstance(mask_names, str):
        mask = data['id'][mask_names]
    else:
        mask = torch.zeros_like(data['id'].y, dtype=torch.bool)
        for name in mask_names:
            mask |= data['id'][name]

    preds_masked = preds[mask]
    labels_masked = labels[mask]
    probs = out[mask][:, 1]  # Fixed: out is a tensor

    acc = accuracy_score(labels_masked.cpu(), preds_masked.cpu())
    precision = precision_score(labels_masked.cpu(), preds_masked.cpu(), zero_division=0)
    recall = recall_score(labels_masked.cpu(), preds_masked.cpu(), zero_division=0)

    try:
        auc = roc_auc_score(labels_masked.cpu(), probs.detach().cpu())
    except ValueError:
        auc = 0.0

    return acc, precision, recall, auc




def train(model, data, optimizer, epochs=100):
    for epoch in range(1, epochs + 1):
        model.train()
        optimizer.zero_grad()

        out = model(data.x_dict, data.edge_index_dict)
        mask = data['id'].train_mask
        #out_id = out['id']
        loss = F.cross_entropy(out[mask], data['id'].y[mask])
        #loss = cross_entropy(out_id[data['id'].train_mask], data['id'].y[data['id'].train_mask])
        loss.backward()
        optimizer.step()

        # Train metrics
        acc, precision, recall, auc = evaluate(model, data, 'train_mask')
        print(f"[Epoch {epoch:03d}] Train - Acc: {acc:.4f} | Prec: {precision:.4f} | Recall: {recall:.4f} | AUC: {auc:.4f}")

        # Val metrics every 10 epochs
        if epoch % 10 == 0:
            acc_val, prec_val, recall_val, auc_val = evaluate(model, data, 'val_mask')
            print(f"[Epoch {epoch:03d}] Val   - Acc: {acc_val:.4f} | Prec: {prec_val:.4f} | Recall: {recall_val:.4f} | AUC: {auc_val:.4f}")

    print("\nFinal Evaluation (Val + Test):")
    acc_final, prec_final, recall_final, auc_final = evaluate(model, data, ['val_mask', 'test_mask'])
    print(f"[Final] Val+Test - Acc: {acc_final:.4f} | Prec: {prec_final:.4f} | Recall: {recall_final:.4f} | AUC: {auc_final:.4f}")




In [8]:

model = HAN(dim_in=-1, dim_out=2)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data, model = data.to(device), model.to(device)

In [9]:
train(model, data, optimizer, epochs=200)


[Epoch 001] Train - Acc: 0.7159 | Prec: 0.1368 | Recall: 0.0511 | AUC: 0.5638
[Epoch 002] Train - Acc: 0.7559 | Prec: 0.0606 | Recall: 0.0064 | AUC: 0.6245
[Epoch 003] Train - Acc: 0.7744 | Prec: 0.2857 | Recall: 0.0064 | AUC: 0.6391
[Epoch 004] Train - Acc: 0.7766 | Prec: 0.0000 | Recall: 0.0000 | AUC: 0.6528
[Epoch 005] Train - Acc: 0.7766 | Prec: 0.0000 | Recall: 0.0000 | AUC: 0.6549
[Epoch 006] Train - Acc: 0.7766 | Prec: 0.0000 | Recall: 0.0000 | AUC: 0.6578
[Epoch 007] Train - Acc: 0.7766 | Prec: 0.0000 | Recall: 0.0000 | AUC: 0.6613
[Epoch 008] Train - Acc: 0.7766 | Prec: 0.0000 | Recall: 0.0000 | AUC: 0.6632
[Epoch 009] Train - Acc: 0.7766 | Prec: 0.0000 | Recall: 0.0000 | AUC: 0.6666
[Epoch 010] Train - Acc: 0.7766 | Prec: 0.0000 | Recall: 0.0000 | AUC: 0.6726
[Epoch 010] Val   - Acc: 0.5548 | Prec: 0.0000 | Recall: 0.0000 | AUC: 0.7136
[Epoch 011] Train - Acc: 0.7766 | Prec: 0.0000 | Recall: 0.0000 | AUC: 0.6808
[Epoch 012] Train - Acc: 0.7766 | Prec: 0.0000 | Recall: 0.0000 

#### Testing Draft Model

In [11]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
#mlflow.set_experiment("spyder-experiment")
import mlflow.pytorch
mlflow.set_experiment("HAN 2025-06-02 Charlie Hebdo")

<Experiment: artifact_location='/workspaces/rumour-detection-gnn/mlruns/62', creation_time=1748915497164, experiment_id='62', last_update_time=1748915497164, lifecycle_stage='active', name='HAN 2025-06-02 Charlie Hebdo', tags={}>

In [ ]:
def evaluate_metrics(model, data, mask):
    model.eval()
    with torch.no_grad():
        out = model(data.x_dict, data.edge_index_dict)
        preds = out.argmax(dim=1)
        probs = out[:, 1]  # Probability of class 1

    true = data['id'].y[mask]
    pred = preds[mask]
    prob = probs[mask]

    acc = accuracy_score(true.cpu(), pred.cpu())
    prec = precision_score(true.cpu(), pred.cpu(), average='macro', zero_division=0)
    recall = recall_score(true.cpu(), pred.cpu(), average='macro', zero_division=0)
    try:
        auc = roc_auc_score(true.cpu(), prob.cpu())
    except:
        auc = 0.0

    return acc, prec, recall, auc

for time_cut in range(20, (60 * 24 * 3), 15):
    print(f"\n=== Time Cut: {time_cut} ===")
    processor = Hetero_Data_Processor_Filter_on_Test_since_first_post(file_path_replies, file_path_posts, time_cut=time_cut)
    data = processor.process()


    model = HAN(dim_in=-1, dim_out=2)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    data, model = data.to(device), model.to(device)

    # Compute imbalance
    y_train = data['id'].y[data['id'].train_mask].cpu()
    imbalance = (y_train == 1).sum() / len(y_train)

    with mlflow.start_run(run_name=f"time_cut_{time_cut}"):
        for epoch in range(1, 101):
            model.train()
            optimizer.zero_grad()
            out = model(data.x_dict, data.edge_index_dict)
            mask = data['id'].train_mask
            loss = F.cross_entropy(out[mask], data['id'].y[mask])
            loss.backward()
            optimizer.step()

            if epoch % 100 == 0:
                 train_acc, train_prec, train_recall, train_auc= evaluate_metrics(model, data, data['id'].train_mask)
                 print(f"[Epoch {epoch}] Train Loss: {loss:.4f} | Train Recall: {train_recall:.4f} | Train Auc: {train_auc:.4f}")
        mlflow.log_metric("train_recall", train_recall, step=epoch)
        mlflow.log_metric("train_precision", train_prec, step=epoch)
        mlflow.log_metric("train_auc", train_auc, step=epoch)

        # Final evaluation on val + test
        final_mask = data['id'].val_mask | data['id'].test_mask
        acc, prec, recall, auc = evaluate_metrics(model, data, final_mask)
        print(f"[Final Val+Test] Acc: {acc:.4f} | Prec: {prec:.4f} | Recall: {recall:.4f} | AUC: {auc:.4f}")

        mlflow.log_metric("final_acc", acc)
        mlflow.log_metric("final_precision", prec)
        mlflow.log_metric("final_recall", recall)
        mlflow.log_metric("final_auc", auc)
        mlflow.log_metric("train_imbalance_rate", imbalance.item())

        mlflow.log_metric("time_cut", time_cut)



=== Time Cut: 20 ===


/workspaces/rumour-detection-gnn/utils.py:1083: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_val_reply_features[['reply_followers','time_diff']] = scaler_replies.transform(test_val_reply_features[['reply_followers','time_diff']])


[Epoch 100] Train Loss: 0.2661 | Train Recall: 0.8304 | Train Auc: 0.9358
[Final Val+Test] Acc: 0.7500 | Prec: 0.8333 | Recall: 0.7500 | AUC: 0.8750

=== Time Cut: 35 ===


/workspaces/rumour-detection-gnn/utils.py:1083: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_val_reply_features[['reply_followers','time_diff']] = scaler_replies.transform(test_val_reply_features[['reply_followers','time_diff']])


[Epoch 100] Train Loss: 0.2883 | Train Recall: 0.8155 | Train Auc: 0.9244
[Final Val+Test] Acc: 0.7200 | Prec: 0.7569 | Recall: 0.7403 | AUC: 0.8701

=== Time Cut: 50 ===


...

/workspaces/rumour-detection-gnn/utils.py:1083: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_val_reply_features[['reply_followers','time_diff']] = scaler_replies.transform(test_val_reply_features[['reply_followers','time_diff']])


[Epoch 100] Train Loss: 0.3073 | Train Recall: 0.8050 | Train Auc: 0.9108
[Final Val+Test] Acc: 0.8625 | Prec: 0.7837 | Recall: 0.7131 | AUC: 0.9100

=== Time Cut: 1175 ===


/workspaces/rumour-detection-gnn/utils.py:1083: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_val_reply_features[['reply_followers','time_diff']] = scaler_replies.transform(test_val_reply_features[['reply_followers','time_diff']])


[Epoch 100] Train Loss: 0.2839 | Train Recall: 0.8297 | Train Auc: 0.9259
[Final Val+Test] Acc: 0.8542 | Prec: 0.7652 | Recall: 0.6992 | AUC: 0.9104

=== Time Cut: 1190 ===


/workspaces/rumour-detection-gnn/utils.py:1083: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_val_reply_features[['reply_followers','time_diff']] = scaler_replies.transform(test_val_reply_features[['reply_followers','time_diff']])
